In [ ]:
#%matplotlib inline

%load_ext autoreload
%autoreload 2

import time
import os
import logging
from functools import wraps
from copy import deepcopy
import dill
import sys
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tqdm.autonotebook import tqdm, trange

from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


# torch packages
import torch
import torch.optim as optim
from torch.nn.functional import mse_loss
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets,transforms
from torchvision.transforms import v2
from torchvision.utils import make_grid
from torchsummary import summary

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils
#from torch.masked import masked_tensor, as_masked_tensor

#DMD packages
from pydmd import DMD
from pydmd.plotter import plot_eigs, plot_summary
#from pydmd.preprocessing import hankel_preprocessing


FILENAME = 'DLDMD_SIC_v1'

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("logs/"+FILENAME+"_" + time.strftime("%Y%m%d-%H%M%S")+ ".txt"),
        #logging.StreamHandler()  #logs to console as well
    ]
)

# Create a logger
logger = logging.getLogger(__name__)

#import custom modules
sys.path.append(r'../src/modules/')

from plot_jupyter import contour_compare, contour_data
from data_wrangle import get_days_before, get_test_set, window_mean
#from dmd_routines import reshape_data2dmd, train_dmd, reshape_Psi2data, eval_dmd

In [ ]:
#seed for repeatable results
torch.manual_seed(0)

gen1 = torch.Generator().manual_seed(0)

In [ ]:
#Import the SIC data set
with open(r'C:/Project/data/data314_years_1989_2023.pkl', 'rb') as f:
    m, Y, Y_mean_month, Y_mean_week, x2, y2 = dill.load(f)

# mask = array of T/F plotting out the land mass of Antartica

mask = ~m  
DATA = Y
x = x2
y = y2
del Y_mean_month, Y_mean_week

In [ ]:
#check data with a plot

# fig_1, ax_1 = plt.subplots(figsize=(5,4))

# ax_1.contourf(DATA[30][350], cmap=plt.get_cmap('Blues_r'))
# ax_1.set_ylim(314, 0)

# plt.show()

#plt.imshow(DATA[30][350], interpolation='nearest')

In [ ]:
def iiee_loss (c_m, c_o, c_e = 0.15):

    model_edge  = c_m >= c_e
    observed_edge = c_o >= c_e

    a_pos = torch.logical_and(model_edge, ~observed_edge)
    a_neg = torch.logical_and(~model_edge, observed_edge)

    a_pos_area = torch.sum(a_pos)
    a_neg_area = torch.sum(a_neg)


    iiee = a_pos_area + a_neg_area
    bias = a_pos_area - a_neg_area

    return iiee, bias, a_pos_area, a_neg_area

In [ ]:
# predicted = torch.Tensor(DATA[30][350])

# observed = torch.Tensor(DATA[30][349])

# iiee_loss(predicted, observed)

In [ ]:
#step parameter
step_thin = 3

# ~ is a binary operator flips 1 to 0 and vice versa

mask = ~m[::step_thin, ::step_thin]

x = x2[::step_thin]
y = y2[::step_thin]

nx = len(x)
ny = len(y)

In [ ]:
#reducing the data set size with the step paramter only for model searching/training
DATA = []
for year in range(len(Y)):
    y0 = Y[year][:, ::step_thin, ::step_thin]
    DATA.append(y0)

DATA[0].shape

In [ ]:
# #check thinned data with a plot
# plt.contourf(DATA[30][350], cmap=plt.get_cmap('Blues_r'))
# plt.show()

# #plt.imshow(DATA[30][350], interpolation='nearest')

In [ ]:
# year within data set
year = 25

# day within year as the starting point 
day = 0

# window-averaging
window = 1

num_year_data = 25

# how many days worth of data needed
T_train = 365 * num_year_data

#batch size for dataloader
batchSize = 420

X0_ = get_days_before(DATA, year, day, T_train+window-1)


#numpy array to torch tensor
all_data = torch.Tensor(X0_)[:,None,:,:]

mask_tensor = torch.Tensor(np.tile(mask, (X0_.shape[0],1,1,1))).bool()

# train_count = 20*365
# val_split = 5*365


# data_max ,data_min, data_avg = all_data.max(), all_data.min(), all_data.mean()

#all_data_train, all_data_val = torch.utils.data.random_split(all_data, [train_count, val_split], generator=gen1)

In [ ]:
# a =

In [ ]:
X1_ = get_test_set(DATA, year, day, window, 365*2)

all_test_data = np.array(X1_)
all_test_data = torch.Tensor(all_test_data)[:,None,:,:]

In [ ]:
#train and test dataloaders

train_loader_sic = torch.utils.data.DataLoader(dataset=all_data, batch_size=batchSize, shuffle=True)

#train_loader_sic = torch.utils.data.DataLoader(dataset=all_data_train, batch_size=batchSize, shuffle=True)

#val_loader_sic  = torch.utils.data.DataLoader(dataset=all_data_val, batch_size=batchSize, shuffle=False)

test_loader_sic = torch.utils.data.DataLoader(dataset=all_test_data, batch_size=1000, shuffle=False)

In [ ]:
class SlideWindowDataset(Dataset):
    def __init__(self, data, window):
        self.data = data
        self.window = window

    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        return x

    def __len__(self):
        return len(self.data) - self.window

In [ ]:
sic_dataset = SlideWindowDataset(all_data[:-365,...], 730)

sic_dataset_pred = SlideWindowDataset(all_data[365:,...], 365)

In [ ]:
train_loader_sic_window = torch.utils.data.DataLoader(dataset=sic_dataset, batch_size=100, shuffle=False)

train_loader_sic_window_pred = torch.utils.data.DataLoader(dataset=sic_dataset_pred, batch_size=100, shuffle=False)

In [ ]:
#A = next(iter(train_loader_sic_window))

In [ ]:
# fig, (ax, ax1) = plt.subplots(1,2, figsize=(20,7))

# contourf_ = ax.contourf(A[0][1], cmap=plt.get_cmap('Blues_r'))
# cbar = fig.colorbar(contourf_)

# contourf_1 = ax1.contourf(A[1][0], cmap=plt.get_cmap('Blues_r'))
# cbar1 = fig.colorbar(contourf_1)

In [ ]:
def dmd_training_snapshots( snapshots, n_prediction_snapshots):
    if n_prediction_snapshots > 0:
        return snapshots[: -n_prediction_snapshots,...]
    return snapshots

def dmd_prediction_snapshots( snapshots,n_prediction_snapshots):
    if n_prediction_snapshots > 0:
        return snapshots[-n_prediction_snapshots :,...]
    return torch.zeros(0)

In [ ]:
# img = all_data[0]


# maxpool = nn.MaxPool2d(4)

# img = nn.Conv2d(1, 4, kernel_size=2)(img)
# img = nn.Dropout2d(p=0.1)(img)
# img = nn.ReLU()(img)
# img = nn.Conv2d(4, 8, kernel_size=3 ,stride=2)(img)
# img = nn.Dropout2d(p=0.1)(img)
# img = nn.ReLU()(img)
# img = nn.Conv2d(8, 16, kernel_size=3, stride=3)(img)
# img = nn.Dropout2d(p=0.1)(img)
# img = nn.ReLU()(img)
# img = nn.Conv2d(16, 32, kernel_size=2, stride=2)(img)
# img = nn.Dropout2d(p=0.1)(img)
# img = nn.ReLU()(img)
# img = nn.Conv2d(32, 64, kernel_size=2)(img)
# #img = maxpool(img)
# img = nn.Flatten()(img[None,:,:])
# img = nn.Linear(3136, 64)(img)

In [ ]:
class Encoder_sharp(nn.Module):
    
    def __init__(self, d_latent = 64):
        """ 
        Encoder using `nn.Module`. Series of 2d-Convolutions, dropouts and Relus

        """
        super().__init__()
        self.d_latent = d_latent
        self.layers = nn.Sequential(
                                        # nn.Conv2d(1, 8, kernel_size=3),
                                        # nn.ReLU(),
                                        #nn.MaxPool2d(2),
                                        nn.Conv2d(1, 16, kernel_size=3),
                                        nn.ReLU(),
                                        nn.MaxPool2d(3),
                                        nn.BatchNorm2d(16),
                                        nn.Conv2d(16, 32, kernel_size=3),
                                        nn.ReLU(),
                                        nn.Conv2d(32, 64, kernel_size=3),
                                        nn.ReLU(),
                                        nn.MaxPool2d(2),
                                        nn.BatchNorm2d(64),
                                        # nn.Conv2d(64, 64, kernel_size=3),
                                        # nn.ReLU(),
                                        # nn.MaxPool2d(2),
                                        # nn.BatchNorm2d(64),
                                        nn.Flatten(),
                                        nn.Linear(160000, self.d_latent)
                                    )
        
        self.layers2 = nn.Sequential(
                                    nn.Linear(self.d_latent, 160000),
                                    nn.ReLU(),
                                    nn.Unflatten(1, (64, 50, 50)),
                                    nn.ConvTranspose2d(64, 16, kernel_size=3,stride = 3),
                                    nn.BatchNorm2d(16),
                                    nn.ConvTranspose2d(16, 8, kernel_size=3, stride = 2),
                                    nn.BatchNorm2d(8),
                                    nn.ConvTranspose2d(8, 4, kernel_size=3),
                                    nn.Conv2d(4, 1, kernel_size=2, padding = 6)
                                    #nn.Conv2d(1, 1, kernel_size=1))
                                    )    
    def forward(self, X):
        h1 = self.layers(X)
        #h1 = self.layers2(h1)
        return h1

In [ ]:
class Decoder_sharp(nn.Module):
    
    def __init__(self,d_latent = 64):
        """ 

        """
        super().__init__()
        self.d_latent = d_latent
        self.layers = nn.Sequential(
                                    nn.Linear(self.d_latent, 160000),
                                    nn.ReLU(),
                                    nn.Unflatten(1, (64, 50, 50)),
                                    nn.ConvTranspose2d(64, 16, kernel_size=3,stride = 3),
                                    nn.BatchNorm2d(16),
                                    nn.ConvTranspose2d(16, 8, kernel_size=3, stride = 2),
                                    nn.BatchNorm2d(8),
                                    nn.ConvTranspose2d(8, 4, kernel_size=3),
                                    nn.Conv2d(4, 1, kernel_size=2, padding = 6)
                                    #nn.Conv2d(1, 1, kernel_size=1))
                                    )    
        #self.layernom = nn.LayerNorm()
    def forward(self, Z):
        h1 = self.layers(Z)
        return h1

In [ ]:
class Encoder(nn.Module):
    
    def __init__(self, d_latent = 64):
        """ 
        Encoder using `nn.Module`. Series of 2d-Convolutions, dropouts and Relus

        """
        super().__init__()
        self.d_latent = d_latent
        self.layers = nn.Sequential(
                                        nn.Conv2d(1, 8, kernel_size=3),
                                        #nn.Dropout2d(p=0.1),
                                        nn.ReLU(),
                                        nn.Conv2d(8, 16, kernel_size=3, stride=2),
                                        #nn.Dropout2d(p=0.1),
                                        nn.ReLU(),
                                        nn.Conv2d(16, 32, kernel_size=3, stride=2),
                                        #nn.Dropout2d(p=0.1),
                                        nn.ReLU(),
                                        nn.Conv2d(32, 64, kernel_size=3),
                                        nn.Flatten(),
                                        nn.Linear(33856, self.d_latent)
                                    )
    def forward(self, X):
        h1 = self.layers(X)
        return h1
    
    #try 5, 3,3 

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self,d_latent = 64):
        """ 

        """
        super().__init__()
        self.d_latent = d_latent
        self.layers = nn.Sequential(
                                    nn.Linear(self.d_latent, 33856),
                                    # 400
                                    nn.ReLU(),
                                    #nn.Linear(28224, 40000),
                                    # 4000
                                    #nn.ReLU(True),
                                    nn.Unflatten(1, (64, 23, 23)),
                                    # 10 x 20 x 20
                                    nn.ConvTranspose2d(64, 16, kernel_size=3, stride = 3),
                                    # 24 x 24
                                    nn.BatchNorm2d(16),
                                    nn.ConvTranspose2d(16, 8, kernel_size=3, stride = 2),
                                    nn.ConvTranspose2d(8, 4, kernel_size=3),
                                    nn.ConvTranspose2d(4, 1, kernel_size=3,padding =19),
                                    nn.BatchNorm2d(1)
                                    )
       
        #self.layernom = nn.LayerNorm()
    def forward(self, Z):
        h1 = self.layers(Z)
        return h1

In [ ]:
class dmd_enc_dec(nn.Module):
    def __init__(self,encoder, decoder, dmd_train, n_prediction_snapshots):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.dmd_train = dmd_train
        self.n_prediction_snapshots = n_prediction_snapshots

    def forward(self, input):

        output_en = self.encoder(input)
        #print(input.shape, output_en.shape)
        self.dmd_train.fit(output_en.T, batch=False)
        self.dmd_train.dmd_time["tend"] = self.dmd_train.original_time["tend"] + self.n_prediction_snapshots
        dmd_recon =  self.dmd_train.reconstructed_data

        if not torch.is_complex(input):
            old_dtype = dmd_recon.dtype
            dmd_recon = dmd_recon.real
            logger.debug(
                f"Removing complex part from output_immersion: {old_dtype} to {dmd_recon.dtype}"
            )
        if dmd_recon.dtype != input.dtype:
            logger.debug(
                f"Casting output_immersion dtype from {dmd_recon.dtype} to {input.dtype}"
            )
            dmd_recon = dmd_recon.to(dtype=input.dtype)

        output = self.decoder(dmd_recon.T)

        return output

In [ ]:
SIC_DMD = DMD(svd_rank=-1)
n_prediction_snapshots = 365

enc = Encoder(d_latent=64)
dec = Decoder(d_latent=64)


if torch.cuda.is_available():
    enc.to("cuda")  
    dec.to("cuda")
    dmddl = dmd_enc_dec(enc,dec, SIC_DMD,n_prediction_snapshots)
    dmddl.to("cuda")  

summary(enc, (1,105,105))

In [ ]:
# enc = Encoder_sharp(d_latent=64)
# dec = Decoder_sharp(d_latent=64)

# if torch.cuda.is_available():
#     enc.to("cuda")  
#     dec.to("cuda")  

# summary(enc, (1,314,314))

**Helper Code**

In [ ]:
#helper online code to auto update activation shapes

# class Network(torch.nn.Module):
#     @staticmethod
#     def calc_activation_shape(
#         dim, ksize, dilation=(1, 1), stride=(1, 1), padding=(0, 0)
#     ):
#         def shape_each_dim(i):
#             odim_i = dim[i] + 2 * padding[i] - dilation[i] * (ksize[i] - 1) - 1
#             return (odim_i / stride[i]) + 1


#         return shape_each_dim(0), shape_each_dim(1)


#     def __init__(self, idim, num_classes=10):
#         self.layer1 = torch.nn.Conv2D(3, 5, 3)
#         ln_shape = Network.calc_activation_shape(idim, 3) # <--- Calculate the shape of output of Convolution
#         self.norm1 = torch.nn.LayerNorm([5, *ln_shape]) # <--- Normalize activations over C, H, and W (see fig.above)
#         self.layer2 = torch.nn.Conv2D(5, 10, 3)
#         ln_shape = Network.calc_activation_shape(ln_shape, 3)
#         self.norm2 = torch.nn.LayerNorm([10, *ln_shape])
#         self.layer3 = torch.nn.Dense(num_classes)


#     def __call__(self, inputs):
#         x = F.relu(self.norm1(self.layer1(input)))
#         x = F.relu(self.norm2(self.layer2(x)))
#         x = F.sigmoid(self.layer3(x))
#         return x


**Helper Code ends**

In [ ]:
a = 

In [ ]:
#### Turn into a function with loss type, normalise type, weight decay, learning rate, print paramters, print at all ####

#### Need to take avg loss and compare for early stopping criteria ####

#### more epochs => significant overfitting ####

#### Retrain on full training set after finding optimal epoch range ####
to_run = True

if to_run: 
    enc.train()
    dec.train()
    dmddl.train()

    # define loss and parameters

    optimizer = optim.Adam(itertools.chain(enc.parameters(), dec.parameters(), dmddl.parameters()), lr=0.0001)
    epochs = 120 
    
    mse_multp = 1.0
    #cls_multp = 0.5

    print('====Training start====')
    for epoch in range(epochs):
        total_reconloss = 0.0
        total_predloss = 0.0
        #train_loss = 0
        for batch_idx, data in enumerate(zip(train_loader_sic_window, train_loader_sic_window_pred)):
            # prepare input data
            #data[0].to("cuda"), data[1].to("cuda") 
            #img_pred = data[1].to("cuda")
            for i in zip(data[0], data[1]):
                img = i[0].to("cuda")
                img_pred = i[1].to("cuda")
                # img.to("cuda")
                # img_pred.to(device='cuda')
            #img = img*mask_tensor
            
                output = dmddl(img)

                output_clamp = output.clamp(min=0, max= 1) #*mask_tensor

                reconstruction_loss = mse_loss(dmd_training_snapshots(output,n_prediction_snapshots), img)

                prediction_loss = mse_loss(dmd_prediction_snapshots(output,n_prediction_snapshots), img_pred)
                
                loss = reconstruction_loss + prediction_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # Track this epoch's loss
                total_reconloss += reconstruction_loss.item()
                total_predloss += prediction_loss.item()

            # if epoch==0 or epoch ==10:    
            #     print(list(enc.parameters())[0]) 
            #total_clsloss += loss_cls.item()

        # if epoch%10==0:    

        #     # add validation step 
        #     enc.eval()
        #     dec.eval()

                #total_val_loss = 0 
            
        #     with torch.no_grad():
        #         for batch_idx_val, data_val in enumerate(val_loader_sic):

        #             img = data.to("cuda")                       
                    
        #             output_en = enc(img)
        #             output = dec(output_en)

        #             output_clamp = output.clamp(min=0, max= 1)#*mask_tensor

        #             #output_norm = (output - output.min())/(output.max() - output.min())

        #             #masked validation
        #             #masked_output = torch.Tensor(masked_tensor(output_clamp, mask_tensor))
        #             #masked_img = torch.Tensor(masked_tensor(img, mask_tensor))

        #             #mask_tensor.detach()

        #             val_loss_mse = distance(img, output_clamp)
                    
        #             #loss_mse = distance(masked_img, masked_output)
        #             #loss_cls = class_loss(output_en, labels)
        #             #loss = (mse_multp * loss_mse) + (cls_multp * loss_cls) 

        #             val_loss = (mse_multp * val_loss_mse) #+ (cls_multp * loss_cls) 
        #             total_val_loss += val_loss
            
        #     enc.train()
        #     dec.train()

            print('====> Epoch: {} Total Recon loss: {:.9f} Total Pred loss: {:.9f}'.format(epoch, total_reconloss,  total_predloss))
            #print(list(enc.parameters())[0]) 
    print('====Training finish====')

    torch.save(enc, 'model_outputs/sic_dmdenc_1_no_mask_latent64_' + time.strftime("%Y%m%d-%H%M%S"))
    torch.save(dec, 'model_outputs/sic_dmddec_1_no_mask_latent64_' + time.strftime("%Y%m%d-%H%M%S"))

In [ ]:
# collect loss and add plot 

# add validation code and plot

In [ ]:
#list(enc.parameters())[0]

In [ ]:
x_train = next(iter(train_loader_sic))[0][None,...]

x_train=x_train.cuda()

latent = enc(x_train)
latent_cpu = enc(x_train).detach().cpu()
plt.plot(latent_cpu.T)
plt.show()

x_train = x_train.detach().cpu().numpy()

recon = dec(latent).clamp(min= 0, max=1)#*mask_tensor
recon = recon.detach().cpu().numpy()

fig, (ax, ax1) = plt.subplots(1,2, figsize=(20,7))

contourf_ = ax.contourf(x_train[0][0], cmap=plt.get_cmap('Blues_r'))
cbar = fig.colorbar(contourf_)

contourf_1 = ax1.contourf(recon[0][0], cmap=plt.get_cmap('Blues_r'))
cbar1 = fig.colorbar(contourf_1)

In [ ]:
x_test = next(iter(test_loader_sic))[100][None,...]
#x_test2 = next(iter(test_loader_sic))[0][None,...]

x_test=x_test.cuda()
#x_test2 = x_test2.cuda()

latent_test = enc(x_test)
latent_test_cpu = enc(x_test).detach().cpu()

plt.plot(latent_test_cpu.T)
plt.show()

x_test = x_test.detach().cpu().numpy()
recon_test = dec(latent_test).clamp(min=0, max =1)#*mask_tensor
recon_test = recon_test.detach().cpu().numpy()

fig1, (ax2, ax3) = plt.subplots(1,2, figsize=(20,7))

contourf_2 = ax2.contourf(x_test[0][0], cmap=plt.get_cmap('Blues_r'))
cbar = fig1.colorbar(contourf_2)

contourf_3 = ax3.contourf(recon_test[0][0], cmap=plt.get_cmap('Blues_r'))
cbar1 = fig1.colorbar(contourf_3)


In [ ]:
X_test_all = next(iter(test_loader_sic))

X_test_all = X_test_all.cuda()#*mask_tensor

enc_test = enc(X_test_all)
dec_test = dec(enc_test).clamp(min=0, max =1 )#*mask_tensor


X_test = X_test_all.permute(1,0,2,3)[0]

dec_test = dec_test.permute(1,0,2,3)[0]

x_true, x_predict = X_test.flatten(), dec_test.flatten()

err = (x_true - x_predict)**2 #/ x_test.shape[0]

total_err =  err.sum().detach().cpu().numpy()
avg_err = total_err/err.shape[0] *100

print(total_err,avg_err)

In [ ]:
## To load models ###

load_ext = True

if load_ext:
    trained_enc  = Encoder_sharp()
    trained_dec = Decoder_sharp()

    trained_enc = torch.load("model_outputs/sic_enc_sharp1_no_mask_latent64_20240611-201106.pt")
    trained_dec = torch.load("model_outputs/sic_dec_sharp1_no_mask_latent64_20240611-201106.pt")
else:
    trained_enc  = enc
    trained_dec = dec


# def load_model(load_ext = False):

#     return trained_e, trained_d

In [ ]:
X0_DMD_train_orig = get_days_before(DATA, 25, day, 365*2+window-1)

X0_DMD_train = torch.Tensor(X0_DMD_train_orig)[:,None,:,:]

train_loader_sic_dmd = torch.utils.data.DataLoader(dataset=X0_DMD_train, batch_size=1000, shuffle=False)

In [ ]:
## Get SVD of dataset ###

U,S, Vh = torch.linalg.svd(torch.Tensor(X0_DMD_train_orig[0]))

# set up temporal range
t_delay = [-i for i in range(730)]

t_delay_fwd = [i for i in range(730)]

t_delay.sort(reverse=False)
t_delay_fwd.sort()

t_delay_all = t_delay + t_delay_fwd

t_delya_arr = np.array(t_delay)

In [ ]:
latent_dmd = DMD(svd_rank=5)

latent_dmd_delay = hankel_preprocessing(latent_dmd, d=100)

latent_bopdmd = BOPDMD(svd_rank=5, eig_constraints={
                                "stable", # choose Re(lambda)<0
                                "conjugate_pairs", # force complex conjugate pairs
                                })


In [ ]:
x_train_dmd = next(iter(train_loader_sic_dmd)) #[:,None,:,:]
x_train_dmd = x_train_dmd.cuda()

In [ ]:
latent_output = trained_enc(x_train_dmd)

latent_output_cpu = latent_output.detach().cpu().numpy().T


# Run standard DMD
latent_dmd.fit(latent_output_cpu)

# DMD modes
Psi_ = latent_dmd.modes
# Get eigenvalues:
Lambda_ = latent_dmd.eigs
# The b_n: IC, expressed in modes basis
bn_ = latent_dmd.amplitudes


# Run BOPDMD #  ---- Need to explain why this is better than DMD use Kutz paper for evidence/rational

latent_bopdmd.fit(latent_output_cpu,t= t_delay)

Psi_bop = latent_bopdmd.modes
# Get eigenvalues:
Lambda_bop = latent_bopdmd.eigs
# The b_n: IC, expressed in modes basis
bn_bop = latent_bopdmd.amplitudes

In [ ]:
# check reconstruction using PYDMD functions  #notbook Autoencoder_SIC

dmd_recon = latent_dmd.reconstructed_data

bopddm_recon = latent_bopdmd.reconstructed_data

plot_summary(latent_dmd)

plot_summary(latent_bopdmd)

In [ ]:
## Plot the latent space and repsective reconstructions 

plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.plot(latent_output_cpu.T)


plt.subplot(1,2,2)
plt.plot(dmd_recon.T)

plt.show()

plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.plot(latent_output_cpu.T)


plt.subplot(1,2,2)
plt.plot(bopddm_recon.T)

plt.show()


In [ ]:
latent_bopdmd_forecast = latent_bopdmd.forecast(t_delay_fwd)

BOPDMDM_forecast_dec = trained_dec(torch.Tensor(latent_bopdmd_forecast.T).cuda())

#BOPDMDM_forecast_dec_norm = (BOPDMDM_forecast_dec - BOPDMDM_forecast_dec.min())/(BOPDMDM_forecast_dec.max() - BOPDMDM_forecast_dec.min())

BOPDMDM_forecast_dec = BOPDMDM_forecast_dec.permute(1,0,2,3)

BOPDMDM_forecast_dec_norm = BOPDMDM_forecast_dec.clamp(min=0, max= 1)

BOPDMDM_forecast_dec_norm = BOPDMDM_forecast_dec_norm.detach().cpu().numpy()

B1 = BOPDMDM_forecast_dec_norm[0][0]

plt.contourf(X0_DMD_train[729][0], cmap=plt.get_cmap('Blues_r'))


plt.figure(figsize=(20,8))
plt.subplot(1,2,1)

x_truth = next(iter(test_loader_sic))[0][0]

plt.contourf(x_truth, cmap=plt.get_cmap('Blues_r'))


plt.subplot(1,2,2)
plt.contourf(B1, cmap=plt.get_cmap('Blues_r'))




In [ ]:
contour_compare(X1_, BOPDMDM_forecast_dec_norm[0])

In [ ]:
#t_all = np.arange(-T_train, T_train)
#true_after = get_test_set(DATA, year, day, window, T_train)
#X_true = np.concatenate((X0, true_after), axis = 0)

#X_pred = eval_dmd(Lambda, Psi, bn, t_all)
# COMPUTE METRIC OF PREDICTION

Integral_pred = np.trapz(np.trapz(BOPDMDM_forecast_dec_norm[0], x, axis = 2), y, axis = 1)
Integral_true = np.trapz(np.trapz(X1_, x, axis = 2), y, axis = 1)

plt.plot(t_delay_fwd, Integral_pred, color = 'grey');
plt.plot(t_delay_fwd, Integral_true, label = 'true total ice', color = 'r')

plt.axvline(0, linestyle = '--', color = 'k')

plt.ylabel('total ice')
plt.xlabel('days')

plt.legend(loc = 'upper right')
# plt.xlim([-120, 10])